# Using Adamalib

This notebook shows an example of using [adamalib]() to control [Adama]().

## Setup

First, define the location of the Adama server.

The official server is: 

> https://api.araport.org/community/v0.3
    
For a development environment the URL may differ (for example: `http://localhost/community/v0.3`).  Use a URL such that `URL/status` reaches the `status` endpoint of your server.

In [1]:
API = 'http://192.168.35.10/community/v0.3'

The official Adama server requires a bearer token, but other test services may not need it. Leave it empty in such case.

In [2]:
TOKEN = 'mytoken'

Create an `adama` object bound to the API server:

In [7]:
import adamalib
adama = adamalib.Adama(API, token=TOKEN)

Check that the object is correctly linked by asking the status of the server. This is equivalent to performing a `GET` request to the endpoint `API/status`.  It should return a dictionary of the form:
```python
{u'api': u'Adama v0.3',
 u'hash': u'...',
 u'status': u'success'}
```

In [8]:
adama.status

{u'api': u'Adama v0.3',
 u'hash': u'195acc791a6b11549da37a59bbd3e010a1e18158',
 u'status': u'success'}

## Namespaces

Namespaces are accessed with the property `namespaces`.  It returns a list of namespace objects, each of them can be queried for its information:

In [9]:
adama.namespaces

[Namespace(foo)]

Let's create a new namespace (we remove it in case it is already created from a previous run).

**Warning**: The following code will destroy a namespace named `foo`. If there is an important namespace with name `foo` and you have permission to delete it, **DO NOT** evaluate the next cells.

In [12]:
adama.foo.delete()
adama.namespaces.add(name='foo')

Namespace(foo)

The new namespace "foo" should appear if we ask again:

In [13]:
adama.namespaces

[Namespace(foo)]

A particular namespace can be accessed directly as:

In [14]:
adama.foo

Namespace(foo)

## Services

List the services contained in a namespace with:

In [15]:
adama.foo.services

[]

### Creating a service

In [31]:
def main(args, adama):
    print(3)

In [34]:
%save goo 31

The following commands were written to file `goo.py`:
def main(args, adama):
    print(3)


In [35]:
ls

adamalib/           CONTRIBUTING.rst     docs/        README.rst
adamalib.egg-info/  dist/                goo.py       requirements.txt
AUTHORS.rst         docker-compose.yml   LICENSE      setup.cfg
bar/                docker-compose.yml~  Makefile     setup.py*
baz/                Dockerfile           MANIFEST.in  tests/
build/              Dockerfile~          notebooks/


In [36]:
cat goo.py

# coding: utf-8
def main(args, adama):
    print(3)


In [37]:
!ls


adamalib	   CONTRIBUTING.rst	docs	     README.rst
adamalib.egg-info  dist			goo.py	     requirements.txt
AUTHORS.rst	   docker-compose.yml	LICENSE      setup.cfg
bar		   docker-compose.yml~	Makefile     setup.py
baz		   Dockerfile		MANIFEST.in  tests
build		   Dockerfile~		notebooks


In [38]:
import os

In [40]:
os.makedirs('tests')

OSError: [Errno 17] File exists: 'tests'

In [41]:
os?